In [ ]:
import json
import copy
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

from coordinate_converter import utm_to_ssb_grid_id

In [ ]:
TRUTHS = "data/incidents_distribution/station_truths.json"
AVERAGE = "data/incidents_distribution/station_avg.json"
PREDICTIONS = "data/incidents_distribution/station_predictions.json"

truths_file = json.load(open(TRUTHS, 'r'))
averages_file = json.load(open(AVERAGE, 'r'))
predictions_file = json.load(open(PREDICTIONS, 'r'))

truths = []
averages = []
predictions = []

for station in range(0, 19):
    for day in range(6, 14):
        for hour in range(0, 24):
            date = datetime(2017, 8, day)
            weekday = date.isocalendar().weekday
            
            truth = truths_file[str(station)][str(2017)][str(8)][str(day)][str(hour)]
            average = averages_file[str(station)][str(8)][str(weekday)][str(hour)]
            prediction = predictions_file[str(station)][str(day)][str(hour)]

            truths.append(truth)
            averages.append(average)
            predictions.append(prediction)

In [ ]:
print("average")
print("mean absolute error:", mean_absolute_error(truths, averages))
print("mean square error:", mean_squared_error(truths, averages))

print("prediction")
print("mean absolute error:", mean_absolute_error(truths, predictions))
print("mean square error:", mean_squared_error(truths, predictions))

print("average")
for i in [(round(p, 3), round(t, 3)) for p, t in zip(averages, truths)]:
    print(i)

print("prediction")
for i in [(round(p, 3), round(t, 3)) for p, t in zip(predictions, truths)]:
    print(i)